In [1]:
import tqdm
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizerFast
from torch.optim import AdamW as TorchAdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
import emoji
import re


import spacy
from bs4 import BeautifulSoup
import gensim
from gensim.models import Word2Vec
import numpy as np
import string
import unidecode
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

2025-03-27 14:38:51.390195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743079131.426733  245091 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743079131.434422  245091 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743079131.481054  245091 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743079131.481071  245091 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743079131.481073  245091 computation_placer.cc:177] computation placer alr

In [2]:
df = pd.read_csv("processed_with_extra_remove_train.csv")
df_test = pd.read_csv("processed_with_extra_remove_test.csv")

# # Pt debug
# df = df.head(10000)
# df_test = df_test.head(10)

# Definesc constante
TEXT_COL = "text"
LABEL_COL = "class"
ID_COL = "id"
device = "cuda"
cols_drop = ["url_count"]

# Drop o col useless
df.drop(columns=cols_drop, inplace=True)
df_test.drop(columns=cols_drop, inplace=True)


In [3]:
len(df_test)

36669

In [4]:
df

,class,text,word_count,unique_word_count,stop_word_count,mean_word_length,char_count,punctuation_count
0,True,psd prabusirea psd altitudinea sigura incertit...,250,180,97,5.196000,1548,38
1,True,amintirea milioane romani beau seara bere blon...,264,178,109,4.481061,1446,42
2,True,cand credea somer gasit lucru credeai ajuns fu...,254,181,108,4.464567,1387,49
3,False,spania goluri echipa nationala romaniei select...,694,416,232,4.965418,4139,189
4,True,campanie veselie cresc campania crescut strada...,116,87,47,4.698276,660,24
...,...,...,...,...,...,...,...,...
70570,True,spatii cazare mare refugiul drumetia echipaj i...,305,218,121,4.881967,1793,86
70571,True,sfantul colantii mall si miorita madalin lasat...,538,360,185,4.842007,3142,118
70572,True,banii sanatatii banii sanatatii facut reforma ...,669,412,282,4.992526,4008,121
70573,True,lege bata premierul tudose premierul mihai tud...,212,132,102,4.353774,1134,30


In [5]:
def drop_outliers(df, th: dict):
    '''
    Functie pentru a elemina toate valorile care sar un threshold setat
    '''
    METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'mean_word_length',
                'char_count', 'punctuation_count']
    
    for i, meta in enumerate(METAFEATURES):
        df = df[(df[meta] > th[meta][0]) & (df[meta] < th[meta][1])]
    return df

# df = drop_outliers(
#     df, 
#     {
#         # Valori determinate folosind ploturile de mai jos
#         "word_count": (0, 2500),
#         "unique_word_count": (0, 1000),
#         "stop_word_count": (0, 600),
#         "mean_word_length": (0, 10),
#         "char_count": (0, 20000),
#         "punctuation_count": (0, 400)
#     }
# )


In [6]:
df_test

,id,title,content,text,word_count,unique_word_count,stop_word_count,mean_word_length,char_count,punctuation_count
0,0,"Financial Times: SUA, Japonia, Australia și In...","SUA, Japonia, Australia și India vor dezvălui ...",financial australia si india lansa sistem moni...,192,131,63,5.786458,1302,26
1,1,Guvernul poate muta Ziua Culturii Române de pe...,Guvernul ia în considerare să mute Ziua Cultur...,guvernul muta ziua culturii romane ianuarie ce...,165,120,54,5.260606,1032,22
2,2,Ministrul ucrainean al Apărării susține că au ...,Aleksei Reznikov a afirmat în cadrul unui mesa...,ministrul ucrainean apararii sustine finalizat...,275,200,96,5.832727,1878,39
3,3,"Amantul unei angajate ANAF s-a sinucis, nemais...",Poliția anchetează cazul unei morți suspecte a...,amantul angajate anaf nemaisuportand numarul d...,189,145,65,5.904762,1304,28
4,4,Boloş: Nu este vorba de o taxă nouă pe concedi...,"„Ca ministru al Finanţelor, este important să ...",vorba taxa concedii eliminare ministru importa...,585,351,254,5.331624,3703,104
...,...,...,...,...,...,...,...,...,...,...
36664,36664,Cei 77 de pompieri români care au ajutat la st...,"Timp de o săptămână, 77 de pompieri români au ...",pompieri romani ajutat stingerea incendiilor t...,209,146,79,5.555024,1369,36
36665,36665,Bill Gates anunță că s-a vaccinat împotriva co...,Fondatorul Microsoft a anunțat chiar el pe Twi...,bill gates anunta vaccinat impotriva beneficii...,310,210,100,5.483871,2009,43
36666,36666,Atelierul auto al Regelui Mihai este deschis p...,"Potrivit Casei Regale, Atelierul Auto a fost i...",atelierul auto regelui mihai deschis publiculu...,267,169,89,5.250936,1668,47
36667,36667,Bibliotecarii cer salarii mai mari. Protest în...,NaN,bibliotecarii cer salarii protest fata ministe...,16,16,3,6.500000,119,3


In [7]:
df["text"] = df["text"].astype(str)

# Fac textele si labelurile ca liste

texts = df["text"].to_list()
labels = df["class"].to_list()

In [8]:
len(df_test["text"].to_list())

36669

In [9]:
len(texts)

70575

In [10]:
labels_one_hot = []

for l in labels: 
    if(l == False): labels_one_hot.append([1, 0])
    else: labels_one_hot.append([0, 1])

labels = labels_one_hot

In [34]:
tokenizer = BertTokenizer.from_pretrained('dumitrescustefan/bert-base-romanian-cased-v1')
model = BertForSequenceClassification.from_pretrained('dumitrescustefan/bert-base-romanian-cased-v1', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Impart datasetul in 3 partie -> train, test, val. Fiecare are 

lim1 = len(texts) // 2
lim2 = lim1 + len(texts) // 5;

train_texts, train_labels = texts[:lim1], labels[:lim1]
test_texts, test_labels = texts[lim1 + 1:lim2], labels[lim1 + 1:lim2]
val_texts, val_labels = texts[lim2 + 1:], labels[lim2 + 1:]

In [13]:
all_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


In [14]:
df_test["text"] = df_test["text"].astype(str)

In [15]:
test_data_encodings = tokenizer(df_test["text"].to_list(), truncation=True, padding=True, max_length=512)
test_data_dataset = torch.utils.data.TensorDataset(torch.tensor(test_data_encodings['input_ids']),
                                               torch.tensor(test_data_encodings['attention_mask']))
test_data_loader = torch.utils.data.DataLoader(test_data_dataset, batch_size=50, shuffle=True)

In [16]:
# Fac un dataset de tensori pentru fiecare set de date.

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(train_labels))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']),
                                             torch.tensor(val_encodings['attention_mask']),
                                             torch.tensor(val_labels))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']),
                                              torch.tensor(test_labels))
all_dataset = torch.utils.data.TensorDataset(torch.tensor(all_encodings['input_ids']),
                                               torch.tensor(all_encodings['attention_mask']),
                                               torch.tensor(labels))


In [17]:
batch_size = 50

# Dataloaderul imaprte datele in segmente de batch_size lungime.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
all_loader = torch.utils.data.DataLoader(all_dataset, batch_size=batch_size, shuffle=True)

In [13]:
# Setez ca parametrii tranable doar cei de classifier, nu cei de bert.

for param in model.bert.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [14]:

optimizer = TorchAdamW(model.classifier.parameters(), lr=1e-5)
model.train()

for epoch in range(1):
    for batch in tqdm.tqdm(train_loader):
        input_ids, attention_mask, labels = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        labels = torch.argmax(labels, dim=1)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), 'BERT_Normal_{i}.pth'.format(i = epoch))

100%|██████████| 706/706 [40:11<00:00,  3.42s/it]


In [28]:
len(test_data_loader)

734

In [ ]:
state_dict = torch.load("BERT_Normal_9.pth")

model.load_state_dict(state_dict)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.63 GiB of which 22.94 MiB is free. Including non-PyTorch memory, this process has 2.85 GiB memory in use. Of the allocated memory 2.59 GiB is allocated by PyTorch, and 203.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [37]:
model.eval()

all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in tqdm.tqdm(all_loader):
       input_ids, attention_mask, labels = batch
       input_ids = input_ids.to(device)
       attention_mask = attention_mask.to(device)
       labels = labels.to(device)

       labels = torch.argmax(labels, dim=1)
       logits = model(input_ids, attention_mask=attention_mask).logits
       all_predictions.extend(logits.tolist())
       all_true_labels.extend(labels.tolist())

  0%|          | 0/1412 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
model.eval()


test_predictions = []

with torch.no_grad():
    for batch in tqdm.tqdm(test_data_loader):
       input_ids, attention_mask = batch
       input_ids = input_ids.to(device)
       attention_mask = attention_mask.to(device)

       logits = model(input_ids, attention_mask=attention_mask).logits
       test_predictions.extend(logits.tolist())


100%|██████████| 734/734 [38:44<00:00,  3.17s/it]


In [ ]:
len(test_predictions)

36669

In [ ]:
bert_test = pd.DataFrame()
bert_test["pred"] = test_predictions

bert_test.to_csv("bert_test.csv", index=False)

In [ ]:
bert_train = pd.DataFrame()
bert_train["pred"] = all_predictions

bert_train.to_csv("bert_train.csv", index=False)